In [1]:
import numpy as np
import numpy.ma as ma
import os
import h5py
from skimage import io
import matplotlib
import matplotlib.pyplot as plt 
import pandas as pd
import math
import time
import sys
import pickle
from visdom import Visdom

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torchsummary import summary
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

from collections import defaultdict
from sklearn.metrics import f1_score
import cv2
import pandas as pd
import h5py

############### Visualization

from ipywidgets import Button, Layout
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display,clear_output,Video
from copy import copy
import numpy.ma as ma
import matplotlib.colors as colors
import imageio


sys.path.insert(0,'/media/DATA/jbonato/astro_segm/Astro3S/modules/')
from gen_single_astro  import filt_im
from sel_active_reg_gen import *
from model.dense_up import dense_up
from test_fun import *

# model to import
model = dense_up(3)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device1 = torch.device('cpu')#('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


###
MAX_ROI_AREA_PROC=40
MU_PX = 1

In [21]:
#PARAMS
N=256
M=256

fov_list = [4]
fov_DNN_weights_folder = '/media/DATA/jbonato/astro_segm/weights/dense_up' 
set_dir='/media/DATA/jbonato/astro_segm/set1/'

####### to rem
from sklearn.cluster import KMeans
class Extr_miniROI():
    
    def __init__(self,Area_mu,mu_px,proc_to_split,split_proc=False):
        
        
        self.Area_mu = Area_mu
        self.mu_px = mu_px
        
        self.Area_px = Area_mu/mu_px**2
        self.proc_to_split = proc_to_split
        self.split_proc = split_proc
    
    @staticmethod
    def det_conn_comp(processes):
        print(processes.shape)
        N,M = processes.shape
        num,comp = cv2.connectedComponents(processes.astype(np.uint8))
        print(num)
        processes_out = np.zeros((num-1,N,M))
        for k in range(1,num):
            print(k)
            pt = np.where(comp==k)
            processes_out[k-1,pt[0],pt[1]]=1
        return processes_out

    def get_k(self,area):
        return area/self.Area_px
    
    def get_miniROI(self):
        if self.split_proc :
            self.proc_to_split = self.det_conn_comp(self.proc_to_split)
        
        Nroi,H,W = self.proc_to_split.shape
        collROI = []
        for i in range(Nroi):
            N = np.sum(self.proc_to_split[i,:,:])
            k = self.get_k(N)
            print(k,int(k))
            if int(k)<=1:
                collROI.append(self.proc_to_split[i,:,:][:,:,np.newaxis])
            else:
                rr = k.astype(np.int64)
                print(rr.dtype)
                buff = np.zeros((H,W,rr))
                pt = np.where(self.proc_to_split[i,:,:]==1)
                X = np.asarray([pt[0],pt[1]]).T
                
                kmeans = KMeans(n_clusters=int(k), random_state=0).fit(X)
                labels = kmeans.labels_
                for j in range(int(k)):
                    pt_lb = np.where(labels==j)
                    buff[X[pt_lb[0],0],X[pt_lb[0],1],j]=1
                collROI.append(buff)
        out = collROI[0]       
        for i in range(len(collROI)):  
            #print(collROI[i].shape)
            out = np.dstack((out,collROI[i]))
        
        return out
    
    
    
def get_signals(roi,stack):
    T,_,_ = stack.shape
    _,_,N = roi.shape
    print(roi.shape)
    signals = []
    for i in range(N-1):
        pt = np.where(roi[:,:,i]==1)
        #ev add single pixels rem
        signals.append(np.mean(stack[:,pt[0],pt[1]],axis=1))
    return signals

def get_signal(roi,stack):
    pt = np.where(roi==1)
    return np.mean(stack[:,pt[0],pt[1]],axis=1)
    

In [22]:
class SimDataset_test(Dataset):
    def __init__(self,image_set):
        self.input_images = image_set    


    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):        
        image = self.input_images[idx]
        image = torch.from_numpy(image).float()

        return image

    
vis_flag = False
##### LOAD PARAM DICT
with open(set_dir+'.tmp/dict_dataset1.txt', "rb") as fp:   #Pickling
    dict_param = pickle.load(fp)
max_min = dict_param['max_min']
dict_param


{'list': [0, 40, 80, 120, 160],
 'blocks': 15,
 'threads': 32,
 'BPM_ratio': 3,
 'bb': 96,
 'N_pix_st': 100,
 'astr_min': 80,
 'percentile': 80,
 'pad': 5,
 'astro_num': 4,
 'init_th_': 0.6,
 'decr_dim': 10,
 'decr_th': 25,
 'corr_int': False,
 'gpu_flag': True,
 'max_min': array([345,  89]),
 'th1_p': 0.25,
 'th2_p': 0.1}

In [30]:
dict_im = {}

for jj in fov_list:
    #im_list = []
    Res_1 = np.zeros((N,M,3))

    test_folder_str =str(jj)
    if len(test_folder_str)==1:
        test_folder_str1='00'+test_folder_str
    else:
        test_folder_str1='0'+test_folder_str

    model.load_state_dict(torch.load(fov_DNN_weights_folder+test_folder_str1+'D1.pt'))
    
    #collect stack to analyze
    
    stack_dir = '/media/DATA/jbonato/segm_project/set/'+test_folder_str+'/'

    items_stack = os.listdir(stack_dir)

    print(stack_dir + items_stack[0])
    stack = io.imread(stack_dir + items_stack[0]).astype(np.uint16)
    dict_im['t-series_'+test_folder_str1] = stack
    frames,_,_ = stack.shape

    a_reg = sel_active_reg(stack.astype(np.float32),dict_param)
    mask = a_reg.get_mask()
    mask = fix_mask(mask)
    #im_list.append([mask,'Active Regions'])
    dict_im['Active Regions_'+test_folder_str1] = mask
    filter_ = filt_im(stack_dir + items_stack[0],mask,dict_param['bb']-2*dict_param['pad'])
    _, image_to_plot = filter_.create_img()
    # for other dataset spatia_pp methods can be called from filter_, outputs are stack filtered and spatial map enhanced  
    #im_list.append([image_to_plot,'Enhanced'])
    dict_im['Enhanced_'+test_folder_str1] = image_to_plot
    coord_l = filter_.get_instances()
    
    assert coord_l!=0, 'Check Active region extraction module'
    
    
    image_stack = np.empty((len(coord_l),dict_param['bb'],dict_param['bb'])) 

    image_stack,filt_imageL = filter_.save_im()#select the padding val 5 is default
    
    
    image_set = image_stack[:,0,:,:]
    image_set = image_set[:,np.newaxis,:,:]

    imageL_set = image_to_plot*filt_imageL
    imageL_set-=np.mean(imageL_set)
    imageL_set= imageL_set[np.newaxis,np.newaxis,:,:]



    model.eval()   # Set model to evaluate mode
    ######################################## Prob Map
    test_datasetL = SimDataset_test(imageL_set)
    test_loader = DataLoader(test_datasetL, batch_size=15, shuffle=False, num_workers=0)

    inputs = next(iter(test_loader))
    inputs = inputs.to(device)
    pred = model(inputs)
    
    pred_mean = pred.data.cpu().numpy()
    del test_datasetL,test_loader, inputs,pred
    
    mean = pred_mean[0]
    maxim = np.amax(mean,axis=0)
    mean[mean<maxim]=0
        
    prob_mapPL,sm_ent = prob_calc(mean[1,:,:],max_min[0],max_min[1])
    
    #im_list.append([prob_mapPL,'Prob. Map'])
    dict_im['Prob. Map PL_'+test_folder_str1] = prob_mapPL
    #im_list.append([sm_ent,'Prob. Map'])
    dict_im['Prob. Map_'+test_folder_str1] = sm_ent
    ########################################## putative single cell
    test_dataset_S = SimDataset_test(image_set)
    test_loader = DataLoader(test_dataset_S, batch_size=15, shuffle=False, num_workers=0)

    
    pred_mean=[]
    for inputs in test_loader:
        inputs = inputs.to(device)

        pred = model(inputs)
        pred_mean.append(pred.data.cpu().numpy())
        del inputs,pred

    torch.cuda.empty_cache()
    
    for j in range(1,len(pred_mean)):
        pred_mean[0]=np.vstack((pred_mean[0],pred_mean[j]))

    prob_map = np.zeros((N,M,2))

    for i in range(len(coord_l)):
        mean= np.zeros((3,dict_param['bb'],dict_param['bb']))
        mean = pred_mean[0][i,:,:,:].copy()
        
        maxim = np.amax(mean,axis=0)
        mean[mean<maxim]=0
        mean[mean>=maxim]=1

        small_soma = small_soma_to_proc(mean[1,:,:],N = int((2/3)*max_min[1])) ####remove too small somata segmentated
        mean[0,:,:]+=small_soma
        mean[1,:,:]-=small_soma
        
        coord = coord_l[i]
        Res_1[coord[1]:coord[3],coord[0]:coord[2],0] += mean[0,dict_param['pad']:-dict_param['pad'],dict_param['pad']:-dict_param['pad']]
        Res_1[coord[1]:coord[3],coord[0]:coord[2],1] += mean[1,dict_param['pad']:-dict_param['pad'],dict_param['pad']:-dict_param['pad']]



    Res_1[:,:,0] -= Res_1[:,:,1]
    Res_1[Res_1<1]=0
    Res_1[Res_1>0]=1

    soma_f = common_merge(Res_1[:,:,1],sm_ent)
    Res_1[:,:,1]=soma_f

    #remove possible artifacts
    small_soma = small_soma_to_proc(Res_1[:,:,1],int(0.9*max_min[1]),dilation=False)
    Res_1[:,:,1]-=small_soma

    Res_1[:,:,0] = Res_1[:,:,0]-Res_1[:,:,1]
    Res_1[Res_1<1]=0
    Res_1[Res_1>0]=1

    #remove large region classified as soma Area>500
    Res_1_filt,removal = art_rem_large(Res_1[:,:,1],Res_1[:,:,0],N=int(1.15*max_min[0]))
    if removal<2:
        Res_1-=Res_1_filt[:,:,np.newaxis]

    Res_1_filt,removal = art_rem_large(Res_1[:,:,1],Res_1[:,:,0],N=max_min[0])
    if removal<2:
        Res_1-=Res_1_filt[:,:,np.newaxis]


    #remove processes without soma
    Res_1_filt = art_rem(Res_1[:,:,1],Res_1[:,:,0])
    Res_1*=Res_1_filt[:,:,np.newaxis]
    
    #im_list.append([Res_1,'Final Mask'])
    dict_im['Final_Mask_'+test_folder_str1] = Res_1
    print(20*'%')
    #######################################################################################################
    #Visualization of images
    if vis_flag:
        vis = Visdom(port=8097, server="http://localhost",env='inference_plot')
        for key in dict_im.keys():
            image = dict_im[key]
            fig, ax = plt.subplots(figsize=(4,4))
            ax.imshow(image)
            ax.axis('off')
            ax.set_title(key)
            vis.matplot(fig)
            plt.close(fig)
    
    ####
    single_astro_roi = gen_sc_mask(dict_im['Final_Mask_'+test_folder_str1])
    dict_roi={}
    dict_traces={}
    
    dict_im['Cell_num_'+test_folder_str1] = single_astro_roi.shape[0]
    print("ROI NUM",single_astro_roi.shape[0])
    for s_roi_num in range(single_astro_roi.shape[0]):
        constr_split_roi = Extr_miniROI(MAX_ROI_AREA_PROC,MU_PX,single_astro_roi[s_roi_num,:,:,0],True)
        arr_out_proc = constr_split_roi.get_miniROI()
        
        name = str(s_roi_num)
        dict_roi['Soma_'+f'{name:0>3}'] = np.where(single_astro_roi[s_roi_num,:,:,1]==1)
        dict_traces['Soma_'+f'{name:0>3}'] = get_signal(single_astro_roi[s_roi_num,:,:,1],dict_im['t-series_'+test_folder_str1])
        for proc_num in range(arr_out_proc.shape[2]):
            name_proc = str(proc_num)
            dict_roi['Proc_'+f'{name:0>3}'+'_'+f'{name_proc:0>3}'] = np.where(arr_out_proc[:,:,proc_num]==1)
            dict_traces['Proc_'+f'{name:0>3}'+'_'+f'{name_proc:0>3}']  = get_signal(arr_out_proc[:,:,proc_num],dict_im['t-series_'+test_folder_str1])
    
    dict_im['Signals_extr_'+test_folder_str1] = dict_traces
    dict_im['ROI_'+test_folder_str1] = dict_roi
    
    #### for Visualization purposes
    constr_split_roi = Extr_miniROI(MAX_ROI_AREA_PROC,MU_PX,dict_im['Final_Mask_'+test_folder_str1][:,:,0],True)
    list_out = constr_split_roi.get_miniROI()

    list_out = np.dstack((list_out,dict_im['Final_Mask_'+test_folder_str1][:,:,1:]))
    dict_im['Final_Mask_fraction_'+test_folder_str1] = list_out
    
    ####

/media/DATA/jbonato/segm_project/set/4/TSeries-04112019-1605-1257_Ch2__movie_corrected_aligned.tiff
Zones 7
qw 25
186 0.92
184 0.93
263 0.91
309 0.95
qw 1
224 0.93
104 0.89
%%%%%%%%%%%%%%%%%%%%
ROI NUM 5
(256, 256)
4
1
2
3
0.15 0
0.4 0
5.35 5
int64
(256, 256)
4
1
2
3
0.775 0
0.475 0
4.7 4
int64
(256, 256)
5
1
2
3
4
2.05 2
int64
1.45 1
3.525 3
int64
1.7 1
(256, 256)
5
1
2
3
4
4.325 4
int64
0.55 0
0.05 0
0.5 0
(256, 256)
5
1
2
3
4
2.975 2
int64
1.875 1
0.275 0
0.975 0
(256, 256)
19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0.15 0
0.4 0
5.35 5
int64
0.775 0
0.475 0
4.7 4
int64
2.05 2
int64
1.45 1
4.325 4
int64
3.525 3
int64
1.7 1
0.55 0
0.05 0
0.5 0
2.975 2
int64
1.875 1
0.275 0
0.975 0


In [24]:
dict_traces.keys()

dict_keys(['Soma_000', 'Proc_000_000', 'Proc_000_001', 'Proc_000_002', 'Proc_000_003', 'Proc_000_004', 'Proc_000_005', 'Proc_000_006', 'Proc_000_007', 'Soma_001', 'Proc_001_000', 'Proc_001_001', 'Proc_001_002', 'Proc_001_003', 'Proc_001_004', 'Proc_001_005', 'Proc_001_006', 'Soma_002', 'Proc_002_000', 'Proc_002_001', 'Proc_002_002', 'Proc_002_003', 'Proc_002_004', 'Proc_002_005', 'Proc_002_006', 'Proc_002_007', 'Proc_002_008', 'Soma_003', 'Proc_003_000', 'Proc_003_001', 'Proc_003_002', 'Proc_003_003', 'Proc_003_004', 'Proc_003_005', 'Proc_003_006', 'Proc_003_007', 'Proc_003_008', 'Proc_003_009', 'Proc_003_010', 'Soma_004', 'Proc_004_000', 'Proc_004_001', 'Proc_004_002', 'Proc_004_003', 'Proc_004_004', 'Proc_004_005', 'Proc_004_006'])

In [25]:
#plt.imshow(np.sum(w[0,:,:,:],axis=2))

In [31]:
def draw_cont(mask):
    N,M,c = mask.shape
    
    sm_c = np.zeros((N,M))
    pr_c = np.zeros((N,M))
    for cnt in range(c-1):
        
        if cnt==c-2:
            
            contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,cnt]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            for j in contours:
                
                sm_c[j[:,0,1],j[:,0,0]]=1
        else:
            
            contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,cnt]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            for j in contours:
                pr_c[j[:,0,1],j[:,0,0]]=1
    pr_c[pr_c>1]=1
    return np.where(sm_c==1),np.where(pr_c==1)#sm_c,pr_c#

def draw_on_Im(image,c_sm,c_pr):
    image_pl = image.copy()
    image_pl = image_pl.astype(np.float64)
    maxim = image_pl.max()
    image_pl/=maxim
    image_pl[c_sm]=4
    image_pl[c_pr]=10
    return image_pl


def get_im(mask,image):
    sm_net,pr_net = draw_cont(mask)
    image_net = draw_on_Im(image,sm_net,pr_net)
    return image_net



def gen_mask_sc(dict_,cell_num,im_shape):
    roi_list = []

    for key in dict_.keys():
        if 'Proc_'+f'{str(cell_num):0>3}'+'_' in key:
            pt = dict_[key]
            buff = np.zeros(im_shape)
            buff[pt[0],pt[1]]=1
            roi_list.append(buff)
    
    pt = dict_['Soma_'+f'{str(cell_num):0>3}']
    buff = np.zeros(im_shape)
    buff[pt[0],pt[1]]=1
    roi_list.append(buff)
    
    buff = np.zeros(im_shape)
    roi_list.append(buff)
    
    out = roi_list[0]
    for i in range(1,len(roi_list)):  
        
        out = np.dstack((out,roi_list[i]))
    return out

def get_f0(traces):
    baseline = np.zeros_like(traces).astype(np.float32)
    for i in range(traces.shape[1]):
        if i<45:
            st=0
        else:
            st=i-45
        if i>frames-45:
            end = -1
        else:
            end=i+45

        baseline[:,i] = np.percentile(traces[:,st:end],20,axis=1)

    return baseline
    
def plot_traces(dict_,cell_num,ax):
    MAX_traces = 7
    
    traces = []
    name=[]
    traces.append(dict_['Soma_'+f'{str(cell_num):0>3}'])
    name.append('Soma')
    cnt=1
    for key in dict_.keys():
        if 'Proc_'+f'{str(cell_num):0>3}'+'_' in key and cnt<= MAX_traces:
            traces.append(dict_[key])
            name.append('Proc.'+str(key[-3:]))
            cnt+=1
    traces = np.asarray(traces)
    print(traces.shape)
    f0 = get_f0(traces)
    traces = (traces-f0)/f0
    delta = 0
    for j in range(MAX_traces):
        ax.plot(np.arange(traces.shape[1]),traces[j,:]+delta)
        ax.text(x = 0, y= traces[j,:].max()/2+delta, s =name[j] , rotation = 0,fontsize= 8)
        delta+= traces[j,:].max()+0.4

In [32]:
fov_name = []
for j in fov_list:
    j = str(j)
    if len(j)==1:
        j='00'+j
    else:
        j='0'+j
    fov_name.append(j)
fov_name

['004']

In [33]:
dict_im['Cell_num_002']

KeyError: 'Cell_num_002'

In [34]:
num= widgets.Dropdown(
    options=fov_name,
    value=fov_name[0],
    description='FOV',
    disabled=False,
)

cell_num= widgets.Dropdown(
    options=[i for i in range(dict_im['Cell_num_'+num.value])],
    value=0,
    description='cell',
    disabled=False,
)

mapped = widgets.Dropdown(
    options=[('inferno',plt.cm.inferno), ('viridis',plt.cm.viridis), ('Greys',plt.cm.gray)],
    value=plt.cm.gray,
    description='Cmap:',
    disabled=False,
)
projection =  widgets.Dropdown(
    options=['median', 'max', 'mean'],
    value='median',
    description='Projection:',
    disabled=False,
)

ROI_type =  widgets.Dropdown(
    options=[('All','_'), ('Fraction','_fraction_')],
    value='_',
    description='Roi Type',
    disabled=False,
)


button = widgets.Button(
    description='Plot',
)
out=widgets.Output(layout=Layout(width='1200px', height='400px'))
out2=widgets.Output(layout=Layout(width='400px', height='400px'))
button=widgets.Button(description='Next')
vbox=widgets.VBox(children=(out,num,cell_num,mapped,projection,ROI_type,button))
hbox=widgets.HBox(children=(vbox,out2))
display(hbox)

def plot2(b=None):
    id_ = num.value    
    stack = dict_im['t-series_'+id_]
    T,N,M = stack.shape
    
    Res_mat = dict_im['Final_Mask'+ROI_type.value+id_]
    video = np.empty((stack.shape[0],256,256),dtype=np.uint8)
    for j in range(stack.shape[0]):
        img = stack[j,:,:]
        bl = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        video[j,:,:]=bl

    imageio.mimwrite('test2.mp4',video, fps=30);
    with out2:
        clear_output(wait=True)
        display(Video('test2.mp4',width=400,height=400))
    if projection.value=='median':
        im = np.median(stack,axis=0)
    elif projection.value == 'mean':
        im = np.mean(stack,axis=0)
    else:
        im = np.amax(stack,axis=0)
    imC = get_im(Res_mat,im)
    
    f, [ax,ax1,ax2] = plt.subplots(1, 3, figsize=(24, 8))
    c = ['snow','lime']
    palette = copy(mapped.value)
    palette.set_over(c[0],1.0)
    palette.set_bad(c[1],1.0)
    zm = ma.masked_where(imC>5,imC)
    ax.imshow(zm,cmap=palette,norm=colors.Normalize(vmin=0,vmax=1.0))
    ax.axis('off')
    
    rois = gen_mask_sc(dict_im['ROI_'+id_],cell_num.value,im.shape)
    im_rois = get_im(rois,im)
    c = ['snow','lime']
    palette = copy(mapped.value)
    palette.set_over(c[0],1.0)
    palette.set_bad(c[1],1.0)
    zm2 = ma.masked_where(im_rois>5,im_rois)
    
    ax1.imshow(zm2,cmap=palette,norm=colors.Normalize(vmin=0,vmax=1.0))
    ax1.axis('off')
    
    plot_traces(dict_im['Signals_extr_'+id_],cell_num.value,ax2)
    

    
    with out:
        clear_output(wait=True)
        display(f)
        
    
button.on_click(plot2)